Control regresion lineal<br>
Sebastian Rosales <br>
201473604-5<br>
sebastian.rosales.14@sansano.usm.cl<br>

In [ ]:
#cargamos las librerias
install.packages('lmtest') #Librería Durbin Watson
install.packages('fBasics') #Librería Kolmogorov-Smirnov

library(lmtest)
library(fBasics)

1) <br>
a) Dado el enunciado, necesitamos predecir cómo se comportará nuestra variable dependiente, en este caso el Dinero, respecto a la variable independiente, la Edad.


In [7]:
#cargamos la data
data = read.csv("DataFormaD.csv")
#visualizamos la informacion
head(data)
#vemos cuantas columnas trae el dataset
nrow(data)

,Edad,Dinero
,<int>,<int>
1,32,98554
2,36,114118
3,26,87184
4,24,77722
5,32,102729
6,28,85420


[1] 200

In [8]:
#construimos la regresion lineal
model <- lm(Dinero ~ Edad,data=data)
model


Call:
lm(formula = Dinero ~ Edad, data = data)

Coefficients:
(Intercept)         Edad  
     -743.6       3236.8  


Del modelo lineal construido, los coeficientes que se obtienen son: <br>
$\hat{\beta_1}$ es $3236.8$ <br>
$\hat{\beta_0}$ es $-743.6$ <br>

El modelo sería de la siguiente forma:<br>
$$ \hat{Y} = 3236.8 \cdot X - 743.6$$



2)<br>
b) **Falsa (F)** El test utilizado para verificar que los errores (residuos) siguen una distribución Normal es el de Kolmogorov-Smirnov

c) **Falsa (F)** No es posible atribuir a una persona de 53 años una cantidad de dinero a gastar, dado que este valor escapa del dominio utilizado para construir la regresión lineal. <br>

Como se muestra a continuación la edad varía desde 18 a 45. 

In [12]:
summary(data)


      Edad           Dinero      
 Min.   :18.00   Min.   : 55555  
 1st Qu.:26.00   1st Qu.: 82688  
 Median :32.00   Median :103246  
 Mean   :31.93   Mean   :102591  
 3rd Qu.:38.00   3rd Qu.:122476  
 Max.   :45.00   Max.   :148030  

d) **Falsa (F)** Al aplicar el Test de ANOVA con $\alpha = 0.05$, se concluye que no existe una relación lineal en el modelo considerando que el estadístico tiene por valor $f=14525$ <br>

**Rechazamos. No hay evidencia suficiente para suponer que el modelo no es lineal.**

Ademas $F_{(1 - \alpha, 1, n-2)} = 3.888$
Observacion: Se utilizo la tabla F de Fisher con el valor en grados de libertad  $n-2 = 198$, calculada online https://www.danielsoper.com/statcalc/calculator.aspx?id=4


In [13]:
fm = aov(model)
summary(fm)

             Df    Sum Sq   Mean Sq F value Pr(>F)    
Edad          1 1.212e+11 1.212e+11   14525 <2e-16 ***
Residuals   198 1.652e+09 8.341e+06                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

e) **Verdadera (V)** Considerando el modelo lineal, el intervalo de confianza para la pendiente tiene por limites $3183.8$ y $3289.7$

Se observa el cálculo de los intervalos de confianza para la pendiente a continuación con $\alpha = 0.05$:

In [14]:
confint(model)

,2.5 %,97.5 %
(Intercept),-2481.720,994.5708
Edad,3183.822,3289.7485


f) **Falsa (F)** No es posible determinar que siguen el comportamiento esperado para un modelo de regresión lineal ya que es necesario aplicarles una transformación ( y concluir a partir de lo obtenido al aplicarla) 

g) **Falsa (F)**. Considerando que el estadístico tiene por valor $D = 0.515$ , y utilizando el test de Kolmogorov-Smirnov con $\alpha = 0.05$. Buscando en la tabla tenemos que $D^{0.05}_{200}$ es $0.096$. Nuestro estadístico $D$ es mayor que $D^{0.05}_{200}$.<br> 

**No hay evidencia suficiente para suponer que los residuos siguen una distribución Normal** 

In [18]:
ksnormTest(fm$residuals) # los residuos se obtienen desde el test de ANOVA


Title:
 One-sample Kolmogorov-Smirnov test

Test Results:
  STATISTIC:
    D: 0.515
  P VALUE:
    Alternative Two-Sided: < 2.2e-16 
    Alternative      Less: < 2.2e-16 
    Alternative   Greater: < 2.2e-16 

Description:
 Mon Aug 10 15:17:25 2020 by user: 


h) **Falsa (F)** Considerando que el estadístico tiene por valor $D = 1.9157$ y utilizando el test de Durbin-Watson con $\alpha = 0.05$, tenemos $dL = 1.758$ y $dU = 1.779$<br> 

Como $D = 1.9157 > d_U = 1.779$, **con un nivel de certeza del 95% los errores no están correlacionados.**

In [20]:
dwtest(model)


	Durbin-Watson test

data:  model
DW = 1.9157, p-value = 0.2738
alternative hypothesis: true autocorrelation is greater than 0
